In [294]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import openpyxl
import geopandas as gpd

In [295]:
input_path='../../data/input/dhis2/new_system/'
output_path='../../data/output/sprint3_analysis/'
shapes_path='../../data/shapes/district/districts_17_19.shp'
input_path_old='../../data/input/dhis2/old_system/'

In [296]:
df_Jan = pd.read_csv(input_path+'Data_elements_(Cases)_jan.csv')
df_Feb = pd.read_csv(input_path+'Data_elements_(Cases)_feb.csv')
df_Mar = pd.read_csv(input_path+'Data_elements_(Cases)_march.csv')
df_Apr = pd.read_csv(input_path+'Data_elements_(Cases)_april.csv')

In [297]:
districts = df_Feb[df_Feb['Data']=='033B-CD01a. Malaria (diagnosed)  - Cases']['Organisation unit']

In [304]:
def data_Quality(indicator):
    df = pd.DataFrame(districts).reset_index(drop=True)

    Jan = df_Jan[df_Jan['Data']==indicator].drop('Data', axis=1)
    Feb = df_Feb[df_Feb['Data']==indicator].drop('Data', axis=1)
    March = df_Mar[df_Mar['Data']==indicator].drop('Data', axis=1)
    April = df_Apr[df_Apr['Data']==indicator].drop('Data', axis=1)
    
    #print(Jan.shape[0]+ Feb.shape[0]+ March.shape[0] + April.shape[0])

    df_final = Jan.merge(Feb,on='Organisation unit', how='outer').merge(March,on='Organisation unit', how='outer').merge(April,on='Organisation unit', how='outer')
    #df_final.columns = ["Districts", "Jan_"+indicator, "Feb_"+indicator, "Mar_"+indicator, "Apr_"+indicator]
    df_final.set_index('Organisation unit',drop=True, inplace=True)
    columns = [('2020',indicator,"Jan"), ('2020',indicator,"Feb"),('2020',indicator,"Mar"),('2020',indicator,"Apr")]
    df_final.columns=pd.MultiIndex.from_tuples(columns)
    return df_final


In [299]:
data_Quality('033B-CD01a. Malaria (diagnosed)  - Cases')

2020                   \
                    033B-CD01a. Malaria (diagnosed)  - Cases                    
                                                         Jan      Feb     Mar   
Organisation unit                                                               
Amuria District                                       2025.0   6710.0  7877.0   
Amuru District                                        5622.0   9317.0  8246.0   
Sironko District                                       398.0   1964.0  1868.0   
Kitgum District                                       1944.0   6135.0  6420.0   
Kitagwenda District                                    537.0   2732.0  1926.0   
...                                                      ...      ...     ...   
Karenga District                                        46.0   1614.0  1607.0   
Mayuge District                                       2625.0  12442.0  9096.0   
Moroto District                                       1285.0   3315.0  1734.0   
Sembabule District                                    3671.0   4672.0  2309.0   
Amudat District                                          NaN   1328.0   568.0   

                              
                              
                         Avr  
Organisation unit             
Amuria District       9388.0  
Amuru District       10237.0  
Sironko District      1841.0  
Kitgum District       7846.0  
Kitagwenda District   2765.0  
...                      ...  
Karenga District      1470.0  
Mayuge District       7701.0  
Moroto District       2439.0  
Sembabule District    2545.0  
Amudat District       1388.0  

[135 rows x 4 columns]

# Data elements

In [308]:
multi_index=pd.MultiIndex.from_tuples([('Test','test','test')])
data_elements=pd.DataFrame(index=districts,columns=multi_index)

In [309]:
indicators = df_Jan['Data'].unique()
for indicator in indicators:
    df_months = data_Quality(indicator)
    data_elements = pd.merge(data_elements, df_months, how='left',left_index=True,right_index=True) 

In [310]:
data_elements

Test                                              2020  \
                     test 033B-TB04. New and Relapse TB cases started on TX   
                     test                                               Jan   
Organisation unit                                                             
Amuria District       NaN                                               NaN   
Sironko District      NaN                                               6.0   
Kitgum District       NaN                                               2.0   
Kitagwenda District   NaN                                               NaN   
Kaberamaido District  NaN                                               3.0   
...                   ...                                               ...   
Ntungamo District     NaN                                               NaN   
Bududa District       NaN                                               2.0   
Karenga District      NaN                                               NaN   
Mayuge District       NaN                                               NaN   
Sembabule District    NaN                                               NaN   

                                                                            \
                                       033B-RV07. RE Blister of 28 tablets   
                       Feb   Mar   Avr                                 Jan   
Organisation unit                                                            
Amuria District        4.0   8.0   2.0                                 NaN   
Sironko District       9.0   7.0   NaN                                29.0   
Kitgum District       33.0  16.0  24.0                               792.0   
Kitagwenda District   10.0  13.0   5.0                                 NaN   
Kaberamaido District   6.0   NaN   1.0                               458.0   
...                    ...   ...   ...                                 ...   
Ntungamo District     37.0  26.0  42.0                                46.0   
Bududa District        7.0   8.0   6.0                              8420.0   
Karenga District       NaN   6.0   3.0                                 NaN   
Mayuge District       11.0  26.0  24.0                                 NaN   
Sembabule District     1.0   9.0  12.0                              1632.0   

                                                   \
                                                    
                          Feb       Mar       Avr   
Organisation unit                                   
Amuria District         470.0     782.0    3740.0   
Sironko District      10312.0    7973.0   11770.0   
Kitgum District       14510.0   12564.0   80269.0   
Kitagwenda District    1400.0    1036.0    2815.0   
Kaberamaido District   1209.0    1488.0    1232.0   
...                       ...       ...       ...   
Ntungamo District     66951.0  121424.0   57038.0   
Bududa District        5604.0   96190.0  158584.0   
Karenga District        713.0    1150.0    1993.0   
Mayuge District       11615.0    6412.0   12580.0   
Sembabule District     5329.0   10678.0    7385.0   

                                                     ...  \
                     033B-CD02a. Dysentery  - Cases  ...   
                                                Jan  ...   
Organisation unit                                    ...   
Amuria District                                 NaN  ...   
Sironko District                                NaN  ...   
Kitgum District                                39.0  ...   
Kitagwenda District                             NaN  ...   
Kaberamaido District                            NaN  ...   
...                                             ...  ...   
Ntungamo District                               NaN  ...   
Bududa District                                 NaN  ...   
Karenga District                                NaN  ...   
Mayuge District                                 NaN  ...   
Sembabule District          

In [311]:
data_elements.drop('test',axis=1,inplace=True,level=2)

In [312]:
null_sum1= data_elements.count()/135

# Indicators 

In [315]:
df_Jan = pd.read_csv(input_path+'Indicator_January.csv')
df_Feb = pd.read_csv(input_path+'Indicator_Feb.csv')
df_Mar = pd.read_csv(input_path+'Indicator_march.csv')
df_Apr = pd.read_csv(input_path+'Indicator_April.csv')

In [316]:
districts = df_Feb[df_Feb['Data']=='EPI - BCG doses given']['Organisation unit'].unique()
multi_index=pd.MultiIndex.from_tuples([('Test','test','test')])

In [317]:
data_Indicators=pd.DataFrame(index=districts,columns=multi_index)

In [318]:
indicators = df_Jan['Data'].unique()
for indicator in indicators:
    df_months = data_Quality(indicator)
    data_Indicators = pd.merge(data_Indicators, df_months, how='left',left_index=True,right_index=True)

In [319]:
data_Indicators.reset_index(inplace=True)
data_Indicators=data_Indicators.groupby('index',axis=0).mean() # This shouldnt be needed, there might be a bug

C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [320]:
null_sum2= data_Indicators.count()/135

# Putting the data together and matching district to shapefiles

In [324]:
# Looking at data quality and null data in one place

null_sum=pd.concat([null_sum1,null_sum2])
null_sum_df = pd.DataFrame(null_sum)
null_sum_df.columns = ['% of data available']
null_sum_df.index.rename(['year','indicator','month'],inplace=True)
null_av_df=null_sum_df.groupby('indicator')['% of data available'].mean()


In [325]:
#Putting both our data together
data=pd.merge(data_Indicators,data_elements,how='inner',left_index=True,right_index=True)

In [326]:
# Using the same name format as in our shapefile

data.rename(index=lambda s: s [:-9],inplace=True)
data.rename(index=str.upper,inplace=True)

In [327]:
data.head()

2020                 \
         105- Total Linked to HIV care (1-4 Yrs) - Female                  
                                                      Jan  Feb  Mar  Avr   
ABIM                                                  NaN  0.0  0.0  2.0   
ADJUMANI                                              1.0  1.0  NaN  0.0   
AGAGO                                                 0.0  0.0  0.0  1.0   
ALEBTONG                                              NaN  NaN  1.0  NaN   
AMOLATAR                                              0.0  NaN  1.0  NaN   

                                                                        \
         105- Total Linked to HIV care (1-4 Yrs) - Male                  
                                                    Jan  Feb  Mar  Avr   
ABIM                                                NaN  0.0  0.0  NaN   
ADJUMANI                                            1.0  0.0  3.0  1.0   
AGAGO                                               0.0  2.0  1.0  0.0   
ALEBTONG                                            1.0  NaN  NaN  NaN   
AMOLATAR                                            2.0  NaN  NaN  1.0   

                                                                 ...  \
         105- Total Linked to HIV care (10-14Yrs) - Female       ...   
                                                       Jan  Feb  ...   
ABIM                                                   NaN  NaN  ...   
ADJUMANI                                               0.0  0.0  ...   
AGAGO                                                  0.0  1.0  ...   
ALEBTONG                                               0.0  1.0  ...   
AMOLATAR                                               0.0  NaN  ...   

                                                      \
         033B-CD14d. Hepatitis B- Cases Positive       
                                             Mar Avr   
ABIM                                         NaN NaN   
ADJUMANI                                     NaN NaN   
AGAGO                                        NaN NaN   
ALEBTONG                                     NaN NaN   
AMOLATAR                                     NaN NaN   

                                                                \
         033B-CD13d. Typhoid Fever- Cases Positive               
                                               Jan Feb Mar Avr   
ABIM                                           NaN NaN NaN NaN   
ADJUMANI                                       NaN NaN NaN NaN   
AGAGO                                          NaN NaN NaN NaN   
ALEBTONG                                       NaN NaN NaN NaN   
AMOLATAR                                       NaN NaN NaN NaN   

                                                                        
         033B-HI11b. Adverse Drug Reactions (ADR) - Deaths              
                                                       Jan Feb Mar Avr  
ABIM                                                   NaN NaN NaN NaN  
ADJUMANI                                               NaN NaN NaN NaN  
AGAGO                                                  NaN NaN NaN NaN  
ALEBTONG                                               NaN NaN NaN NaN  
AMOLATAR                                               NaN NaN NaN NaN  

[5 rows x 1444 columns]

In [328]:
# Modifying the names to extacly fit whats in the shapefile

shapes = gpd.read_file(shapes_path)

In [329]:
# Creating a dict of names to replce (two slighly diffferent spellings )

def get_district_name_dict (df):
    keys = list(set(df.index).difference(set(shapes['name_19'])))
    values = list(set(shapes['name_19']).difference(set(df.index)))
    district_name = dict(zip(keys, values))
    return district_name 


In [330]:
data.rename(index=get_district_name_dict(data),inplace=True)

print(list(set(data_target.index).difference(set(shapes['name_19']))))

[]


# Selecting both relevant and well reported indicators

In [127]:
# Relevant ones, based on my manual selection

target = pd.read_excel(input_path+'Data summary.xlsx',sheet_name='target')

In [187]:
# Well reported ones, based on number of nulls

#null_av_df.to_csv(output_path+'null_values_summary.csv')

with ExcelWriter(input_path+'Data summary.xlsx',mode='a') as writer:
   null_av_df.to_excel(writer,sheet_name='null_values_summary')

In [128]:
#both

target = pd.merge(target,null_av_df,left_on='Var',right_on='indicator',how='left')
target['keep']=(target['% of data available']>0.7)

In [130]:
target_indicators=list(target.loc[target['keep']==1,'Var'])


# Look at EPI data

In [197]:
epi_target=['EPI - BCG doses given',
            'EPI - BCG to MR 1 dropout rate (%)',
            'EPI - DPT 1 to DPT3 dropout rate (%)',
            #'EPI - HPV1 doses given',
            #'EPI - HPV2 doses given',
            'EPI - PCV 1 doses_Under 1',
            'EPI - PCV 2 doses_Under 1',
            'EPI - PCV 3 doses_Under 1',
            'EPI - DPT-HepB-HIB 1 doses given',
            'EPI - DPT-HepB-HIB 3 doses given',
            'EPI - MR 1 doses given']
            #'EPI - MR 2 doses given'

#Some excluded due to low or unstable reporting rates

In [331]:
data_target=data.iloc[:, data.columns.get_level_values(1).isin(epi_target)].copy()
#data_target.reset_index(inplace=True)

In [332]:
# Is the delta over 4 month the best measure?

print(null_sum.loc['2020',epi_target,['Jan','Feb','Mar','Apr']])

# Looks like yes, as the reporting rates seem steady over the period 


year  indicator                             month
2020  EPI - BCG doses given                 Jan      1.000000
                                            Feb      1.000000
                                            Mar      1.000000
                                            Avr      1.000000
      EPI - PCV 1 doses_Under 1             Jan      1.000000
                                            Feb      1.000000
                                            Mar      1.000000
                                            Avr      1.000000
      EPI - PCV 2 doses_Under 1             Jan      1.000000
                                            Feb      1.000000
                                            Mar      1.000000
                                            Avr      1.000000
      EPI - PCV 3 doses_Under 1             Jan      1.000000
                                            Feb      1.000000
                                            Mar      1.000000
                    

In [333]:
def get_delta(indicator):
    delta=((data_target[('2020',indicator,'Apr')]
            -data_target[('2020',indicator,'Jan')])
           /data_target[('2020',indicator,'Jan')])
    return delta

In [334]:
for indicator in epi_target:
    data_target[('2020',indicator,'delta_4')]=get_delta(indicator)

Weird phenomenon : negative drop out rates 

"Check for inconsistency in related indicators – **An example of such inconsistency is when there is a negative dropout over a full year** between the number of children receiving the first dose of DPT vaccine and the number receiving the third dose of the vaccine.  Another example, as illustrated in Figure 1 above, is inconsistency between the number of children receiving DPT vaccine and the number of children receiving OPV vaccine.  Yet another example is when there is a negative “wastage rate” (discussed below) -- when the number of children reported to have received a vaccine is greater than the number of doses of the vaccine reported to have been consumed."

from this [WHO document](https://www.who.int/healthinfo/FacilityAnalysisGuide_Immunization.pdf?ua=1#page=11)


In [202]:
data_target.head()

EPI - BCG doses given                        \
                           Jan     Feb    Mar    Avr   
ABIM                     354.0   294.0  342.0  310.0   
ADJUMANI                 976.0  1055.0  866.0  864.0   
AGAGO                    736.0   667.0  670.0  510.0   
ALEBTONG                 809.0   744.0  654.0  656.0   
AMOLATAR                 440.0   544.0  490.0  316.0   

         EPI - PCV 1 doses_Under 1                       \
                               Jan    Feb    Mar    Avr   
ABIM                         336.0  304.0  421.0  356.0   
ADJUMANI                    1157.0  991.0  937.0  836.0   
AGAGO                        744.0  733.0  968.0  575.0   
ALEBTONG                     838.0  786.0  789.0  817.0   
AMOLATAR                     543.0  521.0  612.0  389.0   

         EPI - PCV 2 doses_Under 1         ... EPI - MR 1 doses given  \
                               Jan    Feb  ...                    Avr   
ABIM                         310.0  301.0  ...                  381.0   
ADJUMANI                    1132.0  950.0  ...                  795.0   
AGAGO                        795.0  809.0  ...                  641.0   
ALEBTONG                     798.0  794.0  ...                  682.0   
AMOLATAR                     538.0  537.0  ...                  320.0   

         EPI - BCG doses given EPI - BCG to MR 1 dropout rate (%)  \
                       delta_4                            delta_4   
ABIM                 -0.124294                          -2.000000   
ADJUMANI             -0.114754                          -1.441989   
AGAGO                -0.307065                          -3.089431   
ALEBTONG             -0.189122                          -1.449438   
AMOLATAR             -0.281818                          -0.817308   

         EPI - DPT 1 to DPT3 dropout rate (%) EPI - PCV 1 doses_Under 1  \
                                      delta_4                   delta_4   
ABIM                                -3.137405                  0.059524   
ADJUMANI                             4.000000                 -0.277442   
AGAGO                                2.142857                 -0.227151   
ALEBTONG                            -0.505682                 -0.025060   
AMOLATAR                             0.629630                 -0.283610   

         EPI - PCV 2 doses_Under 1 EPI - PCV 3 doses_Under 1  \
                           delta_4                   delta_4   
ABIM                      0.058065                 -0.133508   
ADJUMANI                 -0.251767                 -0.291771   
AGAGO                    -0.109434                 -0.154321   
ALEBTONG                 -0.011278                 -0.098477   
AMOLATAR                 -0.256506                 -0.304802   

         EPI - DPT-HepB-HIB 1 doses given EPI - DPT-HepB-HIB 3 doses given  \
                                  delta_4                          delta_4   
ABIM                             0.369048                        -0.128947   
ADJUMANI                        -0.287788                        -0.270798   
AGAGO                           -0.247382                        -0.151478   
ALEBTONG                        -0.027381                        -0.101215   
AMOLATAR                        -0.207721                        -0.268041   

         EPI - MR 1 doses given  
                        delta_4  
ABIM                   0.395604  
ADJUMANI              -0.321672  
AGAGO                 -0.019878  
ALEBTONG              -0.074627  
AMOLATAR              -0.308855  

[5 rows x 45 columns]

# Add in the EPI BCG data from DHIS2, as a test

In [245]:
old_bcg=pd.read_csv(input_path_old+'bcg_doses_given.csv')

In [246]:
# Get the data we are interested in, using the right index

old_bcg['district']=old_bcg['organisationunitname'].apply(lambda x: x[:-9].upper())
old_bcg.set_index('district',drop=True,inplace=True)
cols = np.arange(0,4)
old_bcg.drop(old_bcg.columns[cols],axis=1,inplace=True)
old_bcg.rename(index=get_district_name_dict(old_bcg),inplace=True)

In [272]:
# build a small fucntion to split the string column name

def split(strng, sep, occ):
    strng = strng.split(sep)
    return sep.join(strng[occ[1]:]), sep.join(strng[:occ[0]]), sep.join(strng[occ[0]:occ[1]])[:3]

In [280]:
# Get the right column names
occurences =[-2,-1]
separator = " "
cols=old_bcg.columns
new_cols=[]
for col in cols:
    new_cols.append(split(col,separator,occurences))

In [284]:
old_bcg.columns=pd.MultiIndex.from_tuples(new_cols)

In [291]:
def get_delta_old(year):
    delta=((old_bcg[(year,'EPI - BCG doses given','Apr')]
            -old_bcg[(year,'EPI - BCG doses given','Jan')])
           /old_bcg[(year,'EPI - BCG doses given','Jan')])
    return delta

In [292]:
for year in ['2018','2019']:
    old_bcg[(year,'EPI - BCG doses given','delta_4')]=get_delta_old(year)

# Get that to CSV

In [335]:
data_target=pd.merge(data_target,old_bcg,how='inner',left_index=True,right_index=True)
data_target.columns=data_target.columns.map("_".join)

In [336]:
data_target

,2020_EPI - BCG doses given_Jan,2020_EPI - BCG doses given_Feb,2020_EPI - BCG doses given_Mar,2020_EPI - BCG doses given_Avr,2020_EPI - PCV 1 doses_Under 1_Jan,2020_EPI - PCV 1 doses_Under 1_Feb,2020_EPI - PCV 1 doses_Under 1_Mar,2020_EPI - PCV 1 doses_Under 1_Avr,2020_EPI - PCV 2 doses_Under 1_Jan,2020_EPI - PCV 2 doses_Under 1_Feb,...,2018_EPI - BCG doses given_May,2018_EPI - BCG doses given_Jun,2018_EPI - BCG doses given_Jul,2018_EPI - BCG doses given_Aug,2018_EPI - BCG doses given_Sep,2018_EPI - BCG doses given_Oct,2018_EPI - BCG doses given_Nov,2018_EPI - BCG doses given_Dec,2018_EPI - BCG doses given_delta_4,2019_EPI - BCG doses given_delta_4
ABIM,354.0,294.0,342.0,310.0,336.0,304.0,421.0,356.0,310.0,301.0,...,408,396,395,464,422,428,368,437,-0.140449,-0.178218
ADJUMANI,976.0,1055.0,866.0,864.0,1157.0,991.0,937.0,836.0,1132.0,950.0,...,857,816,895,1008,1007,1170,1132,842,-0.097669,-0.157058
AGAGO,736.0,667.0,670.0,510.0,744.0,733.0,968.0,575.0,795.0,809.0,...,945,886,735,604,989,1068,839,780,-0.260045,-0.063786
ALEBTONG,809.0,744.0,654.0,656.0,838.0,786.0,789.0,817.0,798.0,794.0,...,745,629,726,844,819,821,841,717,-0.118758,-0.262615
AMOLATAR,440.0,544.0,490.0,316.0,543.0,521.0,612.0,389.0,538.0,537.0,...,501,561,547,546,590,576,630,497,-0.093264,-0.196429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SOROTI,1242.0,1064.0,1167.0,942.0,1068.0,1061.0,1170.0,933.0,1080.0,1158.0,...,1300,1230,1193,1287,1286,1478,1211,1340,-0.095319,-0.120654
TORORO,1816.0,1571.0,1664.0,1325.0,1914.0,1761.0,1533.0,3733.0,1784.0,1877.0,...,1862,1714,1891,2006,1883,2288,1897,1536,0.060292,-0.026240
WAKISO,8464.0,6552.0,6298.0,3622.0,8377.0,8662.0,7681.0,3815.0,7789.0,8362.0,...,7515,6348,6931,7144,7288,8567,7360,6299,-0.010227,-0.034970
YUMBE,1371.0,1188.0,1410.0,1469.0,2118.0,1915.0,2138.0,1760.0,1902.0,1877.0,...,1981,1623,1731,1775,1822,2400,1870,1594,0.018474,-0.073573


In [337]:
data_target.to_csv(output_path+'epi_deltas.csv')

In [184]:
shapes[shapes['name_19']=='MUKONO']

,region,name_16,name_17,code_17,name_18,code_18,name_19,ADM1_PCODE,geometry
101,NORTH BUGANDA,MUKONO,MUKONO,108,MUKONO,108,MUKONO,UG108,"POLYGON ((32.87618 -0.13956, 32.87619 -0.13964..."
128,NORTH BUGANDA,MUKONO,MUKONO,108,MUKONO,108,MUKONO,UG108,"POLYGON ((32.87619 -0.13964, 32.87618 -0.13956..."


In [1]:
# DOes that change register bruv?